In [1]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification 
from tensorflow.keras.optimizers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

from ordered_set import OrderedSet
import pandas as pd
import numpy as np

2026-02-07 22:30:29.240324: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-07 22:30:29.266083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-07 22:30:29.266110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-07 22:30:29.266838: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-07 22:30:29.271297: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("="*70)
print("CONFIGURACIÓN DE TENSORFLOW 2.15.1 CON CUDA 12.2")
print("="*70)

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"\nGPUs disponibles: {len(gpus)}")
for gpu in gpus:
    print(f"  - {gpu}")

CONFIGURACIÓN DE TENSORFLOW 2.15.1 CON CUDA 12.2

GPUs disponibles: 0


2026-02-07 22:30:34.535837: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-02-07 22:30:34.537687: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import tensorflow as tf

# 1. Comprobar si TensorFlow fue compilado con soporte CUDA
print("¿Soporte CUDA?: ", tf.test.is_built_with_cuda())

# 2. Listar dispositivos físicos (GPUs) detectados
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detectadas: {len(gpus)}")
    for gpu in gpus:
        print(f" - Nombre: {gpu}")
else:
    print("No se detectó ninguna GPU. Revisa los drivers o la versión de CUDA.")

¿Soporte CUDA?:  True
No se detectó ninguna GPU. Revisa los drivers o la versión de CUDA.


### **Carga Datos**

In [4]:
train_df = pd.read_csv('./Kaggle/Go_Emotions/data/train.tsv', sep='\t', header=None, names=['text','labels', 'code'])
validation_df = pd.read_csv('./Kaggle/Go_Emotions/data/dev.tsv', sep='\t', header=None, names=['text','labels', 'code'])
test_df = pd.read_csv('./Kaggle/Go_Emotions/data/test.tsv', sep='\t', header=None, names=['text','labels', 'code'])

print(train_df.shape, validation_df.shape, test_df.shape)


(43410, 3) (5426, 3) (5427, 3)


In [5]:
train_df.drop(inplace=True, axis=1, labels=['code'])
validation_df.drop(inplace=True, axis=1, labels=['code'])
test_df.drop(inplace=True, axis=1, labels=['code'])


In [6]:
decode_labels = {}
i = 0
with open('./Kaggle/Go_Emotions/data/emotions.txt', 'r') as decoding:
    for line in decoding:
        decode_labels[i] = line.strip()
        i += 1
print(decode_labels)

{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}


In [7]:
def decoding(label):
    label = list(map(int, label.split(',')))
    decoded = []
    for x in label:
        decoded.append(decode_labels.get(x))
    return decoded

train_df['labels'] = train_df['labels'].apply(decoding)

In [8]:
validation_df['labels'] = validation_df['labels'].apply(decoding)
test_df['labels'] = test_df['labels'].apply(decoding)

In [9]:
goemotions_to_fer = {
    "anger": "angry",
    "annoyance": "angry",
    "disapproval": "angry", 

    "disgust": "disgust",

    "fear": "fear",
    "nervousness": "fear",

    "admiration": "happy",
    "amusement": "happy",
    "approval": "happy",
    "caring": "happy",
    "excitement": "happy",
    "gratitude": "happy",
    "joy": "happy",
    "love": "happy",
    "optimism": "happy",
    "pride": "happy",
    "relief": "happy",

    "disappointment": "sad",
    "embarrassment": "sad",
    "grief": "sad",
    "remorse": "sad",
    "sadness": "sad",

    "surprise": "surprise",
    "realization": "surprise",
    "curiosity": "surprise",  
    "confusion": "surprise",

    "neutral": "neutral"  
}

In [10]:
def map_to_fer(go_labels, mapping):
    mapped = OrderedSet()
    for label in go_labels:
        if label in mapping:
            mapped.add(mapping[label])
    return list(mapped) if mapped else ["neutral"]  
train_df['fer_labels'] = train_df['labels'].apply(lambda labs: map_to_fer(labs, goemotions_to_fer))
validation_df['fer_labels'] = validation_df['labels'].apply(lambda labs: map_to_fer(labs, goemotions_to_fer))
test_df['fer_labels'] = test_df['labels'].apply(lambda labs: map_to_fer(labs, goemotions_to_fer))

In [11]:
all_labels = train_df['fer_labels'].explode().value_counts()
print(all_labels)

fer_labels
happy       16935
neutral     14608
angry        5579
surprise     5367
sad          3263
disgust       793
fear          726
Name: count, dtype: int64


In [12]:
train_df['fer_label'] = train_df['fer_labels'].apply(lambda x: x[0])
validation_df['fer_label'] = validation_df['fer_labels'].apply(lambda x: x[0])
test_df['fer_label'] = test_df['fer_labels'].apply(lambda x: x[0])

In [13]:
train_df['sentiment_final'] = train_df['labels'].apply(lambda x: x[0])
validation_df['sentiment_final'] = validation_df['labels'].apply(lambda x: x[0])
test_df['sentiment_final'] = test_df['labels'].apply(lambda x: x[0])

In [14]:
map_dict = {
    "angry":0,
    "disgust":1,
    "fear":2,
    "happy":3,
    "sad":4,
    "surprise":5,
    "neutral":6
}

In [15]:
train_df['label_id'] = [map_dict[i] for i in train_df['fer_label']]
validation_df['label_id'] = [map_dict[i] for i in validation_df['fer_label']]
test_df['label_id'] = [map_dict[i] for i in test_df['fer_label']]

num_labels = len(map_dict)
print(num_labels)

7


In [16]:
train_texts = train_df['text'].tolist()
train_labels = train_df['label_id'].tolist()
val_texts = validation_df['text'].tolist()
val_labels = validation_df['label_id'].tolist()
test_texts = test_df['text'].tolist()
test_labels = test_df['label_id'].tolist()

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_texts, val_labels))

train_ds = train_ds.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(8).prefetch(tf.data.AUTOTUNE)

In [18]:
def balance_classes(df, label_col):
  min_count = df[label_col].value_counts().min()
  min_count
  df = (
      df.groupby(label_col, group_keys=False)
        .apply(lambda x: x.sample(min_count, random_state=42))
        .reset_index(drop=True)
  )
  df.head()
  return df

In [19]:
df = pd.concat([train_df, validation_df,test_df], ignore_index=True)

In [20]:
# df = balance_classes(df, 'fer_label')
# df.head()

In [21]:
df = df.drop(columns=['labels', 'fer_labels', 'label_id','sentiment_final'])
df = df.rename(columns={"fer_label": "sentiment_final"})    

df['sentiment_final'].value_counts()    

sentiment_final
happy       20599
neutral     16545
angry        6729
surprise     5593
sad          3264
fear          792
disgust       741
Name: count, dtype: int64

In [22]:
df.head()

,text,sentiment_final
0,My favourite food is anything I didn't have to...,neutral
1,"Now if he does off himself, everyone will thin...",neutral
2,WHY THE FUCK IS BAYLESS ISOING,angry
3,To make her feel threatened,fear
4,Dirty Southern Wankers,angry


### **Entrenamiento**

#### **DistilBERT**

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [24]:
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['sentiment_final'])

# Número de clases
num_labels = len(label_encoder.classes_)
print(f"Número de emociones/clases: {num_labels}")
print(f"Clases: {label_encoder.classes_}")


Número de emociones/clases: 7
Clases: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [25]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values,
    df['label_encoded'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label_encoded']
)

In [26]:
class EmotionDataset(Dataset):
    """Dataset personalizado para clasificación de emociones con DistilBERT"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenizar
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [27]:
print("\n📦 Cargando DistilBERT tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_dataset = EmotionDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer
)

val_dataset = EmotionDataset(
    texts=val_texts,
    labels=val_labels,
    tokenizer=tokenizer
)

print(f"✅ Datasets creados - Train: {len(train_dataset)}, Val: {len(val_dataset)}")


📦 Cargando DistilBERT tokenizer...
✅ Datasets creados - Train: 43410, Val: 10853


In [28]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

print(f"✅ DataLoaders creados - Batches por época: {len(train_loader)}")

✅ DataLoaders creados - Batches por época: 2714


In [29]:
print("\n📦 Cargando modelo DistilBERT preentrenado...")
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=num_labels
)

model.to(device)
print(f"✅ Modelo cargado en {device}")

# Mostrar parámetros del modelo
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Parámetros totales: {total_params:,}")
print(f"📊 Parámetros entrenables: {trainable_params:,}")


📦 Cargando modelo DistilBERT preentrenado...


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


✅ Modelo cargado en cuda
📊 Parámetros totales: 66,958,855
📊 Parámetros entrenables: 66,958,855


#### **Roberta**

In [30]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [32]:
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['sentiment_final'])

# Número de clases
num_labels = len(label_encoder.classes_)
print(f"Número de emociones/clases: {num_labels}")
print(f"Clases: {label_encoder.classes_}")

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values,
    df['label_encoded'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label_encoded']
)

Número de emociones/clases: 7
Clases: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [33]:
class EmotionDataset(Dataset):
    """Dataset personalizado para clasificación de emociones con RoBERTa"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenizar
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [34]:
print("\n📦 Cargando RoBERTa tokenizer...")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_dataset = EmotionDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer
)

val_dataset = EmotionDataset(
    texts=val_texts,
    labels=val_labels,
    tokenizer=tokenizer
)

print(f"✅ Datasets creados - Train: {len(train_dataset)}, Val: {len(val_dataset)}")


📦 Cargando RoBERTa tokenizer...
✅ Datasets creados - Train: 43410, Val: 10853


In [35]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

print(f"✅ DataLoaders creados - Batches por época: {len(train_loader)}")

✅ DataLoaders creados - Batches por época: 2714


In [36]:

print("\n📦 Cargando modelo RoBERTa preentrenado...")
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=num_labels
)

model.to(device)
print(f"✅ Modelo cargado en {device}")

# Mostrar parámetros del modelo
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Parámetros totales: {total_params:,}")
print(f"📊 Parámetros entrenables: {trainable_params:,}")



📦 Cargando modelo RoBERTa preentrenado...


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


✅ Modelo cargado en cuda
📊 Parámetros totales: 124,651,015
📊 Parámetros entrenables: 124,651,015


In [37]:
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Learning rate scheduler con warmup
from transformers import get_linear_schedule_with_warmup

num_epochs = 3
total_steps = len(train_loader) * num_epochs
warmup_steps = total_steps // 10  # 10% de warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

ValueError: Attempt to convert a value (<generator object Module.parameters at 0x7f544c1ebb50>) with an unsupported type (<class 'generator'>) to a Tensor.

#### **Distilroberta**

In [38]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [40]:
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['sentiment_final'])

# Número de clases
num_labels = len(label_encoder.classes_)
print(f"Número de emociones/clases: {num_labels}")
print(f"Clases: {label_encoder.classes_}")

Número de emociones/clases: 7
Clases: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [42]:
print("\n⚠️ NOTA: Emotion-DistilRoBERTa está pre-entrenado para estas emociones:")
print("   anger, disgust, fear, joy, neutral, sadness, surprise")
print("   Si tus clases son diferentes, el modelo se adaptará durante el fine-tuning.")

# Split train/validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values,
    df['label_encoded'].values,
    test_size=0.2,
    random_state=42,
    stratify=df['label_encoded']
)


⚠️ NOTA: Emotion-DistilRoBERTa está pre-entrenado para estas emociones:
   anger, disgust, fear, joy, neutral, sadness, surprise
   Si tus clases son diferentes, el modelo se adaptará durante el fine-tuning.


In [43]:
class EmotionDataset(Dataset):
    """Dataset personalizado para clasificación de emociones con Emotion-DistilRoBERTa"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenizar
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [44]:
print("\n📦 Cargando Emotion-DistilRoBERTa tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('j-hartmann/emotion-english-distilroberta-base')

train_dataset = EmotionDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer
)

val_dataset = EmotionDataset(
    texts=val_texts,
    labels=val_labels,
    tokenizer=tokenizer
)

print(f"✅ Datasets creados - Train: {len(train_dataset)}, Val: {len(val_dataset)}")


📦 Cargando Emotion-DistilRoBERTa tokenizer...


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Datasets creados - Train: 43410, Val: 10853


In [45]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

print(f"✅ DataLoaders creados - Batches por época: {len(train_loader)}")

✅ DataLoaders creados - Batches por época: 2714


In [46]:
print("\n📦 Cargando modelo Emotion-DistilRoBERTa preentrenado...")
model = AutoModelForSequenceClassification.from_pretrained(
    'j-hartmann/emotion-english-distilroberta-base',
    num_labels=num_labels,
    ignore_mismatched_sizes=True  # Permite cambiar el número de clases
)

model.to(device)
print(f"✅ Modelo cargado en {device}")

# Mostrar parámetros del modelo
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Parámetros totales: {total_params:,}")
print(f"📊 Parámetros entrenables: {trainable_params:,}")


📦 Cargando modelo Emotion-DistilRoBERTa preentrenado...


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: j-hartmann/emotion-english-distilroberta-base
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Modelo cargado en cuda
📊 Parámetros totales: 82,123,783
📊 Parámetros entrenables: 82,123,783


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [47]:
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=0.01)

# Learning rate scheduler con warmup
from transformers import get_linear_schedule_with_warmup

num_epochs = 3
total_steps = len(train_loader) * num_epochs
warmup_steps = total_steps // 10  # 10% de warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

print(f"\n⚙️ Configuración de entrenamiento:")
print(f"   Learning rate: {5e-6}")
print(f"   Épocas: {num_epochs}")
print(f"   Total steps: {total_steps}")
print(f"   Warmup steps: {warmup_steps}")

ValueError: Attempt to convert a value (<generator object Module.parameters at 0x7f546c4f4c10>) with an unsupported type (<class 'generator'>) to a Tensor.